In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [103]:
team_stats = pd.read_csv('../data/FS_champ_team_2021.csv')
games = pd.read_csv('../data/FS_champ_games_2021.csv')
goal_timings = pd.read_csv('../data/FS_Champ2021_Goal_timings.csv')

In [104]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [105]:
games = games[0:552]
#Dropping playoff matches

In [106]:
team_stats.columns

Index(['team_name', 'common_name', 'season', 'country', 'matches_played',
       'matches_played_home', 'matches_played_away', 'suspended_matches',
       'wins', 'wins_home',
       ...
       'goals_conceded_min_61_to_70', 'goals_conceded_min_71_to_80',
       'goals_conceded_min_81_to_90', 'draw_percentage_overall',
       'draw_percentage_home', 'draw_percentage_away',
       'loss_percentage_ovearll', 'loss_percentage_home',
       'loss_percentage_away', 'over145_corners_percentage'],
      dtype='object', length=293)

In [107]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [108]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

,common_name,xg_against_avg_overall
0,Stoke City,1.21
1,Middlesbrough,1.13
2,AFC Bournemouth,1.29
3,Swansea City,1.28
4,Watford,1.29


In [109]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [110]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [111]:
fill_stats('Norwich City')
fill_stats('Blackburn Rovers')
fill_stats('Brentford')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Watford')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Rotherham United')
fill_stats('Wycombe Wanderers')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Sheffield Wednesday')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

In [112]:
games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_btts_yes,odds_btts_no,stadium_name,over/under,home_xg,away_xg,home_xg_against,away_xg_against,home_total_corners,away_total_corners
0,1599849900,Sep 11 2020 - 6:45pm,complete,NaN,Watford,Middlesbrough,Keith Stroud,1.0,0.00,0.00,...,1.95,1.74,Vicarage Road (Watford),0,1.36,1.44,1.29,1.13,5.61,5.96
1,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Birmingham City,Brentford,Tony Harrington,1.0,0.00,0.00,...,1.87,1.80,St Andrew's Trillion Trophy Stadium (Birmingham),0,0.94,1.55,1.27,1.12,4.11,4.73
2,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Wycombe Wanderers,Rotherham United,James Linington,1.0,0.00,0.00,...,1.74,1.95,Adams Park (High Wycombe- Buckinghamshire),0,1.32,1.32,1.59,1.37,4.3,5.04
3,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,AFC Bournemouth,Blackburn Rovers,Gavin Ward,1.0,0.00,0.00,...,1.74,1.95,Vitality Stadium (Bournemouth- Dorset),1,1.44,1.75,1.29,1.29,5.21,4.61
4,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Barnsley,Luton Town,Jarred Gillett,1.0,0.00,0.00,...,1.77,1.91,Oakwell (Barnsley- South Yorkshire),0,1.43,1.29,1.08,1.34,4.6,5.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1620473400,May 08 2021 - 11:30am,complete,NaN,Middlesbrough,Wycombe Wanderers,Martin Atkinson,46.0,1.68,0.64,...,1.74,2.00,Riverside Stadium (Middlesbrough),1,1.44,1.32,1.13,1.59,5.96,4.3
548,1620473400,May 08 2021 - 11:30am,complete,NaN,Nottingham Forest,Preston North End,Darren Bond,46.0,1.18,1.45,...,1.71,2.00,The City Ground (Nottingham- Nottinghamshire),1,1.33,1.11,1.39,1.46,5.04,4.11
549,1620473400,May 08 2021 - 11:30am,complete,NaN,Queens Park Rangers,Luton Town,Tony Harrington,46.0,1.55,1.32,...,1.67,2.10,Kiyan Prince Foundation Stadium (London),1,1.47,1.29,1.49,1.34,5.07,5.13
550,1620473400,May 08 2021 - 11:30am,complete,NaN,Reading,Huddersfield Town,Jarred Gillett,46.0,1.77,0.77,...,1.71,2.00,Madejski Stadium (Reading- Berkshire),1,1.36,1.11,1.26,1.44,4.24,4.44


In [113]:
games.corr()['over/under'].sort_values(ascending=False)[0:20]

over/under                             1.000000
total_goal_count                       0.813089
total_goals_at_half_time               0.564385
home_team_goal_count                   0.563420
away_team_goal_count                   0.538042
home_team_goal_count_half_time         0.402322
home_team_shots_on_target              0.361050
away_team_goal_count_half_time         0.358810
away_team_shots_on_target              0.284876
team_a_xg                              0.221940
team_b_xg                              0.181444
home_team_shots                        0.156785
away_team_shots                        0.150139
average_goals_per_match_pre_match      0.091487
over_15_percentage_pre_match           0.084846
over_05_HT_FHG_percentage_pre_match    0.084284
over_05_2HG_percentage_pre_match       0.083594
over_25_percentage_pre_match           0.083280
Game Week                              0.078257
away_team_possession                   0.077439
Name: over/under, dtype: float64

# Now let's do some modeling

In [177]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = games[features]
y = games['over/under']

In [178]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 42)

In [179]:
logreg = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

In [180]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [181]:
preds = logreg.predict(X_test)

In [182]:
accuracy_score(y_test, preds)

0.6428571428571429

In [183]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [184]:
preds = knn.predict(X_test)

In [185]:
accuracy_score(y_test, preds)

0.5714285714285714

In [186]:
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [187]:
preds = dt.predict(X_test)

In [188]:
accuracy_score(y_test, preds)

0.5714285714285714